## Libraries und Vairablen

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
buchhaus_new_last_30 = 'https://www.buchhaus.ch/de/heute/last30'

## Webscraping

In [8]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get(buchhaus_new_last_30)

buch_df = pd.DataFrame(columns=['title',  
                            'preis',
                            'untertitel',
                            'autor', 
                            'verlag',
                            'sprache',
                            'einband',
                            'erscheinungsjahr',
                            'seiten',
                            'artikelnummer',
                            'isbn',
                            'auflage',
                            'reihe',
                            'reihenbandnummer',
                            'genre',
                            'text'])

for page in range(1, 5):
    for book in range(1,5):
        try:
            buch = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[4]/div/div/div[{}]/div/div[1]/div/div/span/span/a'.format(book))
            buch.click()

            # get the data
            try: titel = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/h1/span/span').text
            except: titel = 'nan'

            try: preis = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[3]/div[1]/div/div/span[2]/span').text
            except: preis = 'nan'

            try: untertitel = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[1]/div[2]/div/div[2]/table/tbody/tr[1]/td[2]').text
            except: untertitel = 'nan'

            try: autor = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/div/div/span/span/span/span/a').text
            except: autor = 'nan'

            try: verlag = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[5]/span[2]/div/div/span/span/a').text
            except: verlag = 'nan'

            try: einband = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[4]/span/span').text
            except: einband = 'nan'

            try: erscheinungsjahr = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[7]/span[2]/span').text
            except: erscheinungsjahr = 'nan'

            try: seiten = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[11]/span[2]/span').text
            except: seiten = 'nan'

            try: artikelnummer = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[12]/span[2]/span').text
            except: artikelnummer = 'nan'

            try: isbn = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[2]/span[2]/span').text
            except: isbn = 'nan'

            try: auflage = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[8]/span[2]/span').text
            except: auflage = 'nan'

            try: reihe = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[9]/span[2]/span/a').text
            except: reihe = 'nan'

            try: reihenbandnummer = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[1]/div[10]/span[2]/span').text
            except: reihenbandnummer = 'nan'

            try: genre = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]/div[2]/div[2]/div[2]/span[2]/span/a').text
            except: genre = 'nan'

            # get the 'Buchbeschreibungen'
            try: text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[2]/article[1]/div[2]').text
            except: pass

            sprache = 'Deutsch'

        # Fehlermeldung
        except: 
            print('error mit Buch {} auf Seite {}'.format(book, page))
            continue
    

        df_temp = pd.DataFrame({'title': titel,  
                        'preis': preis,
                        'untertitel': untertitel,
                        'autor': autor, 
                        'verlag': verlag,
                        'sprache': sprache,
                        'einband': einband,
                        'erscheinungsjahr': erscheinungsjahr,
                        'seiten': seiten,
                        'artikelnummer':artikelnummer,
                        'isbn': isbn,
                        'auflage': auflage,
                        'reihe': reihe,
                        'reihenbandnummer': reihenbandnummer,
                        'genre': genre,
                        'text': text
                        }, index=[0])
        buch_df = pd.concat([buch_df, df_temp], ignore_index=True)
    
        #back to the main page
        driver.get(buchhaus_new_last_30)

    # go to the next page
    if page == 1: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a').click()
    else: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a[2]').click()
    time.sleep(1)


C:\Users\yanni\AppData\Local\Temp\ipykernel_16084\147013529.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


## Datawrangling

## Ausblick

nächste Schritte:

* anpassung der Unterachiedlichen Buchinformationen

Ausblick:

* Bücher in den nächsten 30 Tagen könnten auch gescrapet werden